In [1]:
import os
import glob
import re
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta
from sunpy.map import Map, make_fitswcs_header
from sunpy.coordinates import HeliographicCarrington, sun

In [2]:
# --- Configuration ---
INPUT_FOLDER = '/home/arpitkumar/surya_workshop/data/target_dataset/wsa_opt_maps'      # Folder containing .npy files
OUTPUT_FOLDER = '/home/arpitkumar/surya_workshop/data/target_dataset/fulldisk_csv/'      # Folder to save the results
TARGET_RESOLUTION = 0.6            # Arcsec/pixel for output
OUTPUT_SHAPE = (4096, 4096)        # Dimensions of output map

def get_cr_mid_date(cr_number):
    """
    Returns the date corresponding to the middle of a Carrington Rotation.
    Start time + 13.6 days (half a solar rotation).
    """
    # Get start time of the rotation (Longitude 360)
    start_time = sun.carrington_rotation_time(cr_number)
    # Add half a rotation period (~13.6 days) to get the center
    mid_time = start_time + TimeDelta(13.6 * u.day)
    return mid_time

def reproject_speed_to_aia(speed_data, target_date):
    """
    Reprojects a 2D WSA optimized speed map (Carrington) to an Earth-view Full Disk.
    """
    # 1. Handle Orientation: Ensure shape matches (Lat, Lon) for SunPy creation
    # If data is (360, 180) -> it's likely (Lon, Lat), so we transpose to (180, 360)
    if speed_data.shape == (360, 180):
        speed_data = speed_data.T
    
    input_shape = speed_data.shape  # Should be roughly (180, 360)

    # 2. Create Input Header (Carrington Frame)
    # CRITICAL FIX: observer="self" with a defined radius anchors the map in 3D
    carrington_frame = HeliographicCarrington(obstime=target_date, observer="self")
    
    input_coord = SkyCoord(180 * u.deg, 0 * u.deg, 1 * u.R_sun, 
                           frame=carrington_frame)
    
    input_header = make_fitswcs_header(
        input_shape,
        input_coord,
        scale=[1, 1] * u.deg / u.pix,  # Assuming 1 degree/pixel resolution
        projection_code="CAR",
        instrument="WSA+ Optimized"
    )
    
    # Create the Map
    input_map = Map(speed_data, input_header)
    input_map.meta['rsun_ref'] = 6.96e8  # Standard solar radius in meters

    # 3. Define Output Header (Virtual AIA View from Earth)
    target_scale = u.Quantity([TARGET_RESOLUTION, TARGET_RESOLUTION], unit=u.arcsec / u.pix)
    
    # Center the view on Earth at the target date
    ref_coord = SkyCoord(0 * u.arcsec, 0 * u.arcsec, 
                         frame="helioprojective", 
                         obstime=target_date, 
                         observer='earth')

    output_header = make_fitswcs_header(
        OUTPUT_SHAPE,
        ref_coord,
        scale=target_scale,
        projection_code="TAN", 
        instrument=f"Virtual AIA {TARGET_RESOLUTION}"
    )

    output_template = Map(np.zeros(OUTPUT_SHAPE), output_header)

    # 4. Reproject
    # interpolation='linear' (or 'bilinear') is standard and faster than adaptive
    print(f"   Reprojecting to {OUTPUT_SHAPE}...")
    full_disk_map = input_map.reproject_to(output_template.wcs, algorithm='interpolation')
    
    return full_disk_map

def main():
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    # Find all .npy files
    file_list = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.npy")))
    
    if not file_list:
        print(f"No .npy files found in {INPUT_FOLDER}")
        return

    print(f"Found {len(file_list)} maps to process.")

    for i, file_path in enumerate(file_list[:10]):
        filename = os.path.basename(file_path)
        
        # --- 1. Extract CR Number ---
        # Regex looks for "CR" followed by digits (e.g., CR2049)
        match = re.search(r'CR(\d+)', filename)
        
        if not match:
            print(f"[{i+1}] Skipping {filename}: Could not find 'CR' number in name.")
            continue
            
        cr_num = int(match.group(1))
        
        print(f"\n[{i+1}/{len(file_list)}] Processing CR{cr_num} ({filename})...")

        try:
            # --- 2. Calculate Date ---
            # Get the mid-point date of this Carrington Rotation
            target_date = get_cr_mid_date(cr_num)
            print(f"   Target Date (Mid-CR): {target_date.iso}")

            # --- 3. Load Data ---
            speed_data = np.load(file_path)
            # Nan handling: set NaNs to 0 (or min speed) to avoid projection errors
            speed_data = np.nan_to_num(speed_data, nan=0.0)

            # --- 4. Reproject ---
            reprojected_map = reproject_speed_to_aia(speed_data, target_date)

            # --- 5. Save to CSV ---
            # Construct output filename
            output_filename = f"reprojected_wsa_CR{cr_num}.csv"
            output_path = os.path.join(OUTPUT_FOLDER, output_filename)
            
            print(f"   Saving to {output_path}...")
            print("   (Warning: File will be approx 300MB)")
            
            # Save data with 4 decimal places to save some space/time
            np.savetxt(output_path, reprojected_map.data, delimiter=',', fmt='%.4f')
            
            print("   Done.")

        except Exception as e:
            print(f"   Error processing CR{cr_num}: {e}")

    print("\nProcessing Complete.")

if __name__ == "__main__":
    main()

Found 129 maps to process.

[1/129] Processing CR2049 (fittedCR2049_wsa_map.npy)...
   Target Date (Mid-CR): 2006-11-01 03:35:09.625
   Reprojecting to (4096, 4096)...


   Saving to /home/arpitkumar/surya_workshop/data/target_dataset/fulldisk_csv/reprojected_wsa_CR2049.csv...
   (Warning: File will be approx 300MB)
   Done.

[2/129] Processing CR2053 (fittedCR2053_wsa_map.npy)...
   Target Date (Mid-CR): 2007-02-18 10:28:57.248
   Reprojecting to (4096, 4096)...


   Saving to /home/arpitkumar/surya_workshop/data/target_dataset/fulldisk_csv/reprojected_wsa_CR2053.csv...
   (Warning: File will be approx 300MB)
   Done.

[3/129] Processing CR2054 (fittedCR2054_wsa_map.npy)...
   Target Date (Mid-CR): 2007-03-17 18:33:43.355
   Reprojecting to (4096, 4096)...


   Saving to /home/arpitkumar/surya_workshop/data/target_dataset/fulldisk_csv/reprojected_wsa_CR2054.csv...
   (Warning: File will be approx 300MB)
   Done.

[4/129] Processing CR2055 (fittedCR2055_wsa_map.npy)...
   Target Date (Mid-CR): 2007-04-14 02:00:03.380
   Reprojecting to (4096, 4096)...


   Saving to /home/arpitkumar/surya_workshop/data/target_dataset/fulldisk_csv/reprojected_wsa_CR2055.csv...
   (Warning: File will be approx 300MB)
   Done.

[5/129] Processing CR2056 (fittedCR2056_wsa_map.npy)...
   Target Date (Mid-CR): 2007-05-11 08:25:15.722
   Reprojecting to (4096, 4096)...


...

   Saving to /home/arpitkumar/surya_workshop/data/target_dataset/fulldisk_csv/reprojected_wsa_CR2057.csv...
   (Warning: File will be approx 300MB)
   Done.

[7/129] Processing CR2058 (fittedCR2058_wsa_map.npy)...
   Target Date (Mid-CR): 2007-07-04 18:40:18.253
   Reprojecting to (4096, 4096)...


   Saving to /home/arpitkumar/surya_workshop/data/target_dataset/fulldisk_csv/reprojected_wsa_CR2058.csv...
   (Warning: File will be approx 300MB)
   Done.

[8/129] Processing CR2059 (fittedCR2059_wsa_map.npy)...
   Target Date (Mid-CR): 2007-07-31 23:28:27.227
   Reprojecting to (4096, 4096)...


   Saving to /home/arpitkumar/surya_workshop/data/target_dataset/fulldisk_csv/reprojected_wsa_CR2059.csv...
   (Warning: File will be approx 300MB)
   Done.

[9/129] Processing CR2060 (fittedCR2060_wsa_map.npy)...
   Target Date (Mid-CR): 2007-08-28 04:44:25.660
   Reprojecting to (4096, 4096)...


   Saving to /home/arpitkumar/surya_workshop/data/target_dataset/fulldisk_csv/reprojected_wsa_CR2060.csv...
   (Warning: File will be approx 300MB)
   Done.

[10/129] Processing CR2061 (fittedCR2061_wsa_map.npy)...
   Target Date (Mid-CR): 2007-09-24 10:42:09.625
   Reprojecting to (4096, 4096)...


   Saving to /home/arpitkumar/surya_workshop/data/target_dataset/fulldisk_csv/reprojected_wsa_CR2061.csv...
   (Warning: File will be approx 300MB)
   Done.

Processing Complete.
